# Data request through API

### This is a small Python code for the collection of daily and hourly electricity consumption, heating, water and cooling datasets from the city of Helsinki buildings database 

In [1]:
# Import necessary libraries
from matplotlib import pyplot as plt
import warnings
import requests
import json
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import concurrent.futures
import time
from functools import partial
from datetime import datetime
import os

In [2]:
response = requests.get('https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List').text

In [3]:
# Get the list of all available buildings
response

'[{"locationName":"1000 Hakaniemen kauppahalli","propertyName":"1000 Hakaniemen kauppahalli","propertyCode":"091-011-9902-0101"},{"locationName":"1001 Hietalahden kauppahalli","propertyName":"1001 Hietalahden kauppahalli","propertyCode":"091-004-9902-0008"},{"locationName":"1002 Vanha kauppahalli","propertyName":"1002 Vanha kauppahalli","propertyCode":"091-003-9906-0101"},{"locationName":"1037 Vuotalo","propertyName":"1037 Vuotalo","propertyCode":"091-054-0179-0003"},{"locationName":"1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen","propertyName":"1507 Suutarilan monitoimitalo/ala-aste ja Lpk Seulanen","propertyCode":"091-040-0024-0003"},{"locationName":"1508 Monitoimitalo Puustelli","propertyName":"1508 Monitoimitalo Puustelli","propertyCode":"091-033-0232-0003, 091-033-0260-0001"},{"locationName":"1509 Lpk Karvikka ja rppk Lemmikki","propertyName":"1509 Lpk Karvikka ja rppk Lemmikki","propertyCode":"091-038-0055-0010"},{"locationName":"1511 Pauligin huvila","propertyName":"151

In [4]:
response_info = json.loads(response)

In [5]:
type(response_info)

list

In [6]:
### An example of request of a daily data for a specific building

requests.get('https://helsinki-openapi.nuuka.cloud/api/v1/EnergyData/Daily/ListByProperty?Record=LocationName&SearchString=4285%20Kaisaniemen%20ala-aste&ReportingGroup=Electricity&StartTime=2019-01-01&EndTime=2021-12-31').json()

[{'timestamp': '2019-01-01T00:00:00',
  'reportingGroup': 'Electricity',
  'locationName': '4285 Kaisaniemen ala-aste',
  'value': 799.8599999999999,
  'unit': 'kWh'},
 {'timestamp': '2019-01-02T00:00:00',
  'reportingGroup': 'Electricity',
  'locationName': '4285 Kaisaniemen ala-aste',
  'value': 787.14,
  'unit': 'kWh'},
 {'timestamp': '2019-01-03T00:00:00',
  'reportingGroup': 'Electricity',
  'locationName': '4285 Kaisaniemen ala-aste',
  'value': 696.9099999999999,
  'unit': 'kWh'},
 {'timestamp': '2019-01-04T00:00:00',
  'reportingGroup': 'Electricity',
  'locationName': '4285 Kaisaniemen ala-aste',
  'value': 789.18,
  'unit': 'kWh'},
 {'timestamp': '2019-01-05T00:00:00',
  'reportingGroup': 'Electricity',
  'locationName': '4285 Kaisaniemen ala-aste',
  'value': 677.54,
  'unit': 'kWh'},
 {'timestamp': '2019-01-06T00:00:00',
  'reportingGroup': 'Electricity',
  'locationName': '4285 Kaisaniemen ala-aste',
  'value': 687.8899999999999,
  'unit': 'kWh'},
 {'timestamp': '2019-01-0

In [7]:
# A simple code for the acquisition of necessary datasets from NUUKA API

def toDatetime(record):

    timestamp = record['timestamp']
    record['timestamp'] = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S")
    return record

def getData(startTime, endTime , freq="Daily"):
    
    reportingGroups = ["Electricity"]
    #reportingGroups = ["Heat"]
    #reportingGroups = ["Water"]
    #reportingGroups = ["DistrictCooling"]
    
    locationsLink = 'https://helsinki-openapi.nuuka.cloud/api/v1.0/Property/List'
    
    # Initialize api
    locationsRequest = requests.get(locationsLink) 
    # Get data from api
    locationsRequest.raise_for_status() 
    locations = pd.DataFrame.from_dict(locationsRequest.json())
    locations = locations['locationName']
    data = []
    dataLink = (f"https://helsinki-openapi.nuuka.cloud/api/v1.0/"
                f"EnergyData/{freq}/ListByProperty")
          
    # Helper function for concurrency 
    def request(group, location):
        payload = {'StartTime': startTime, 'EndTime': endTime}
        payload.update({'ReportingGroup': group})
        payload.update({'SearchString': location.split(' ', 1)[0]})
        r = requests.get(dataLink, payload)
        if r.status_code == requests.codes.ok:
            data_dict = r.json()
            for record in data_dict:
                toDatetime(record)
            data.extend(data_dict)
    
    # The final step - getting the data.
    for g in reportingGroups:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(partial(request, g), locations)
            
    # For each "thread", execute "request" function on each location
    return pd.DataFrame(data)

### Save data to a local .csv file 


In [32]:
# Request the data from the API.
start = time.perf_counter()
ts = getData('2019-01-01', '2021-12-31', freq = 'Daily') 
finish = time.perf_counter()
print(f'Finished in {finish-start} second(s)')

# Export the dataframe to a .csv file
file_name = 'Helsinki_all_daily.csv' # change this to the respective frequency of the data
ts.to_csv(file_name)

Finished in 98.63750589999836 second(s)


In [8]:
dataset=pd.read_csv('Helsinki_all_daily.csv')

In [9]:
#Please pay attention to the .csv document where IDs can change time to time due to the API updates
df2 = dataset.iloc[345316:346406]

In [10]:
df2.rename(columns = {'value':'ElCons', 'timestamp':'Date'}, inplace = True)
df2.index = df2.Date
df2=df2.drop(['Date'],axis=1)

In [11]:
df2

,Unnamed: 0,reportingGroup,locationName,ElCons,unit
Date,,,,,
2019-01-01,345316,Electricity,4285 Kaisaniemen ala-aste,799.86,kWh
2019-01-02,345317,Electricity,4285 Kaisaniemen ala-aste,787.14,kWh
2019-01-03,345318,Electricity,4285 Kaisaniemen ala-aste,696.91,kWh
2019-01-04,345319,Electricity,4285 Kaisaniemen ala-aste,789.18,kWh
2019-01-05,345320,Electricity,4285 Kaisaniemen ala-aste,677.54,kWh
...,...,...,...,...,...
2021-12-26,346401,Electricity,4285 Kaisaniemen ala-aste,258.54,kWh
2021-12-27,346402,Electricity,4285 Kaisaniemen ala-aste,540.82,kWh
2021-12-28,346403,Electricity,4285 Kaisaniemen ala-aste,526.72,kWh


In [12]:
df2=df2.drop(columns=['Unnamed: 0','reportingGroup','locationName','unit'])

In [13]:
df2.rename(columns = {'value':'ElCons'}, inplace = True)

In [14]:
df2

,ElCons
Date,
2019-01-01,799.86
2019-01-02,787.14
2019-01-03,696.91
2019-01-04,789.18
2019-01-05,677.54
...,...
2021-12-26,258.54
2021-12-27,540.82
2021-12-28,526.72


In [48]:
# Save a file with daily electricity consumption data

#df2.to_csv('kaisa_dailyElCons.csv')                             
 

Weather data can be downloaded manually from https://en.ilmatieteenlaitos.fi/download-observations